In [1]:
import os
import gc
import json
import time
import joblib
import warnings
import traceback
import numpy as np
import pandas as pd
import optuna
import pandas_ta as ta
import tensorflow as tf
from datetime import datetime
import numpy as np
import pandas as pd
import pandas_ta as ta
from scipy.signal import argrelextrema
from numba import jit
from collections import Counter
import numpy as np
import pandas as pd
import optuna
import joblib
import json
import gc
import traceback
import os
from datetime import datetime
import tensorflow as tf
# Machine Learning
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    confusion_matrix, classification_report
)
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler
# Numba (For Fast Backtesting)
from numba import jit
import shap
import numpy as np
import pandas as pd

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# GPU Check
if tf.config.list_physical_devices('GPU'):
    print(f"🚀 GPU Detected: {tf.config.list_physical_devices('GPU')}")
else:
    print("⚠️ No GPU detected. Using CPU.")
    

def remove_raw_prices_and_transform(df, target_type, method):
    df_transformed = df.copy()
    
    # 1. 로그 수익률 생성 (변화 없음)
    if 'eth_log_return' not in df_transformed.columns:
        close_col = 'ETH_Close' if 'ETH_Close' in df_transformed.columns else 'close'
        if close_col in df_transformed.columns:
            df_transformed['eth_log_return'] = np.log(df_transformed[close_col] / df_transformed[close_col].shift(1))
    
    # 2. ATR 변환 (변화 없음)
    if 'ATR_84' in df_transformed.columns:
        close_col = 'ETH_Close' if 'ETH_Close' in df_transformed.columns else 'close'
        if close_col in df_transformed.columns:
            df_transformed['ATR_84'] = (df_transformed['ATR_84'] / df_transformed[close_col]) * 100
            df_transformed['ATR_84'] = df_transformed['ATR_84'].replace([np.inf, -np.inf], 0).fillna(0)

    # --- [신규] 제거 대상 정의 ---
    
    # 1) 원시 가격/거래량 데이터 제거 패턴 (기존 유지)
    raw_price_patterns = [
        '_close', '_open', '_high', '_low', '_volume', 
        'tvl', 'mcap', 'sp500', 'gold', 'dxy', 'vwap' 
    ]
    
    # 2) 살려둘 안전 키워드 (비율, 지표 등)
    keep_keywords = [
        '_pct', '_ratio', '_chg', '_diff', '_dist', '_score', 'zscore', 
        'natr', 'rsi', 'mfi', 'adx', 'cci', 'vr', 'shadow', 
        'div', 'spike', 'climax', 'absorption', 'breakout', 'price_vs',
        'atr_84' 
    ]

    # 3) [핵심] 무조건 제거할 타 알트코인 리스트 (대소문자 무시 매칭용)
    # ETH(타겟)와 BTC(대장)는 제외하고 나머지 다 씀
    drop_coins = ['XRP', 'SOL', 'ADA', 'DOGE', 'AVAX', 'DOT']
    
    cols_to_remove = []
    
    # 필수 보존 컬럼
    essential_cols = ['date', 'target', 'next_open', 'next_direction', 'next_close', 'next_log_return', 
                      'take_profit_price', 'stop_loss_price', 'real_entry_price', 'ATR_84']

    for col in df_transformed.columns:
        if col in essential_cols:
            continue
            
        col_lower = col.lower()
        
        # --------------------------------------------------------------------
        # [A] 타 알트코인 관련 컬럼인지 확인 (지표든 가격이든 무조건 제거)
        # 예: DOGE_MACD, XRP_RSI, ADA_Close ...
        # --------------------------------------------------------------------
        is_altcoin = False
        for coin in drop_coins:
            # coin 이름이 컬럼명에 포함되어 있으면 (예: "DOGE" in "DOGE_MACD")
            # 단, 대소문자 구분 없이 체크 (coin.lower() in col_lower)
            if coin.lower() in col_lower:
                is_altcoin = True
                break
        
        if is_altcoin:
            cols_to_remove.append(col)
            continue # 알트코인이면 아래 로직 볼 필요 없이 즉시 제거 목록 추가

        # --------------------------------------------------------------------
        # [B] 원시 가격 데이터인지 확인 (ETH, BTC라도 원시 가격은 제거)
        # --------------------------------------------------------------------
        is_dangerous = any(p in col_lower for p in raw_price_patterns) or col in ['EMA_72', 'SMA_120', 'OBV']
        is_safe = any(k in col_lower for k in keep_keywords)
        
        # 위험 패턴이 있고 + 안전 키워드가 없으면 제거
        if is_dangerous and not is_safe:
            cols_to_remove.append(col)

    # 중복 제거 후 삭제 실행
    cols_to_remove = list(set(cols_to_remove))
    
    if cols_to_remove:
        # 로그 좀 예쁘게 찍기 (알트코인 제거 개수 파악)
        alt_drop_count = sum(1 for c in cols_to_remove if any(coin.lower() in c.lower() for coin in drop_coins))
        price_drop_count = len(cols_to_remove) - alt_drop_count
        
        print(f"[Feature Cleaning] Dropping {len(cols_to_remove)} columns.")
        print(f"  - Altcoin Metrics: {alt_drop_count} (DOGE, XRP, etc.)")
        print(f"  - Raw Prices/Volumes: {price_drop_count}")
        
        df_transformed.drop(cols_to_remove, axis=1, inplace=True)
        
    return df_transformed


2025-12-03 00:09:52.149513: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-03 00:09:52.149552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-03 00:09:52.150900: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-03 00:09:52.158132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-03 00:09:52.930414: W tensorflow/comp

🚀 GPU Detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from numba import jit

@jit(nopython=True)
def compute_targets_with_hourly_numba_4h(
    base_dates_ts, base_atr, hourly_dates_ts, hourly_open,
    hourly_high, hourly_low, lookahead_periods, profit_mult, stop_mult
):
    n = len(base_dates_ts)
    
    # [수정 1] targets 배열을 float64로 초기화하고 기본값을 np.nan으로 설정
    # 정수형(int)에는 NaN이 없으므로 float로 변경해야 함
    targets = np.full(n, np.nan, dtype=np.float64)
    
    entry_prices = np.full(n, np.nan, dtype=np.float64)
    tp_prices = np.full(n, np.nan, dtype=np.float64)
    sl_prices = np.full(n, np.nan, dtype=np.float64)
    
    four_hour_ms = 14400000
    lookahead_ms = lookahead_periods * four_hour_ms
    
    h_start = 0
    
    for i in range(n - 1):
        atr = base_atr[i]
        if np.isnan(atr) or atr <= 0: 
            continue
        
        entry_start_ts = base_dates_ts[i] + four_hour_ms
        entry_end_ts = entry_start_ts + lookahead_ms
        
        first_entry_idx = -1
        for h in range(h_start, len(hourly_dates_ts)):
            if hourly_dates_ts[h] >= entry_start_ts:
                first_entry_idx = h
                h_start = h 
                break
        
        if first_entry_idx == -1:
            continue
        
        entry_price = hourly_open[first_entry_idx]
        tp = entry_price + (atr * profit_mult)
        sl = entry_price - (atr * stop_mult)
        
        entry_prices[i] = entry_price
        tp_prices[i] = tp
        sl_prices[i] = sl
        
        # [수정 2] result 초기값을 np.nan으로 설정 (기존 0)
        result = np.nan 
        
        # 타임아웃 여부 확인용 플래그
        is_timeout = False 
        
        for h in range(first_entry_idx, len(hourly_dates_ts)):
            # 1. 시간 초과 체크
            if hourly_dates_ts[h] >= entry_end_ts:
                is_timeout = True
                result=0.0
                break
            
            # 2. 손절 (Stop Loss) -> 0.0 (실패)
            if hourly_low[h] <= sl:
                result = 0.0
                break
                
            # 3. 익절 (Take Profit) -> 1.0 (성공)
            if hourly_high[h] >= tp:
                result = 1.0
                break
        

        targets[i] = result
    
    return targets, entry_prices, tp_prices, sl_prices


def create_targets_4h(df_4h, df_1h, lookahead=30, profit_mult=1.5, stop_mult=1.0, **kwargs):
    df_target = df_4h.copy()
    hourly_df = df_1h.copy()
    
    # 날짜 변환
    df_target['date'] = pd.to_datetime(df_target['date'])
    hourly_df['datetime'] = pd.to_datetime(hourly_df['datetime'])
    
    hourly_df.columns = hourly_df.columns.str.lower()
    
    # ATR 없으면 계산
    if 'ATR_84' not in df_target.columns:
        df_target['ATR_84'] = ta.atr(
            df_target['ETH_High'], df_target['ETH_Low'], df_target['ETH_Close'], length=84
        )
    
    df_target = df_target.sort_values('date').reset_index(drop=True)
    hourly_df = hourly_df.sort_values('datetime').reset_index(drop=True)
    
    # Numba 입력용 변환
    base_dates_ts = df_target['date'].astype(np.int64).values // 10**6
    base_atr = df_target['ATR_84'].fillna(method='ffill').fillna(0).to_numpy()
    
    hourly_dates_ts = hourly_df['datetime'].astype(np.int64).values // 10**6
    hourly_open = hourly_df['open'].astype(np.float64).to_numpy()
    hourly_high = hourly_df['high'].astype(np.float64).to_numpy()
    hourly_low = hourly_df['low'].astype(np.float64).to_numpy()
    
    # 타겟 계산 (Numba)
    targets, entry_prices, tp_prices, sl_prices = compute_targets_with_hourly_numba_4h(
        base_dates_ts, base_atr, hourly_dates_ts, hourly_open, hourly_high, hourly_low,
        lookahead, profit_mult, stop_mult
    )
    
    df_target['next_direction'] = targets
    df_target['real_entry_price'] = entry_prices
    df_target['take_profit_price'] = tp_prices
    df_target['stop_loss_price'] = sl_prices
    
    # [핵심] next_log_return 계산 (dropna 하기 전에 수행!)
    # 다음 봉의 종가와 시가를 이용해 로그 수익률 계산 (Shift -1)
    df_target['next_close'] = df_target['ETH_Close'].shift(-1)
    df_target['next_open'] = df_target['ETH_Open'].shift(-1)
    
    # 0으로 나누기 방지용 epsilon 추가
    df_target['next_log_return'] = np.log(df_target['next_close'] / (df_target['next_open'] + 1e-9))
    
    # lookahead 만큼 뒤쪽 데이터는 결과 확인 불가하므로 제거
    if lookahead > 0:
        df_target = df_target.iloc[:-lookahead]
    
    valid_before = len(df_target)
    
    df_target = df_target.dropna(subset=['next_direction']).reset_index(drop=True)
    

    df_target['next_log_return'] = df_target['next_log_return'].fillna(0.0)
    
    # 타겟을 int로 변환
    df_target['next_direction'] = df_target['next_direction'].astype(int)
    
    valid_after = len(df_target)
    print(f"Valid Samples: {valid_after}/{valid_before} (Dropped: {valid_before - valid_after})")
    
    return df_target




In [3]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor

# [추가됨] 상관관계가 높은 피처를 제거하는 함수
def remove_collinear_features(X, threshold=0.9):
    """
    상관관계 계수가 threshold 이상인 피처들을 찾아서
    둘 중 하나(뒤에 나오는 컬럼)를 제거합니다.
    """
    # 수치형 데이터만 선택하여 상관계수 행렬 계산
    corr_matrix = X.corr().abs()
    
    # 상삼각 행렬(Upper Triangle)만 선택 (대각선 제외)
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # threshold보다 높은 상관계수를 가진 컬럼 이름 추출
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    if to_drop:
        print(f"  [Collinearity] Dropping {len(to_drop)} features with correlation > {threshold}")
    
    return X.drop(columns=to_drop), to_drop

def select_features_verified(X_train, y_train, task='class', top_n=20, verbose=True):
    # 1. [추가됨] 먼저 상관관계가 너무 높은 피처를 제거 (여기서 먼저 걸러내는 것이 효율적)
    X_train_filtered, dropped_cols = remove_collinear_features(X_train, threshold=0.90)
    
    if verbose:
        print(f"Features reduced from {X_train.shape[1]} to {X_train_filtered.shape[1]} after correlation check.")

    # 이후 로직은 필터링된 X_train_filtered를 사용
    if len(X_train_filtered) > 10000:
        idx = np.random.choice(len(X_train_filtered), 10000, replace=False)
        X_sub = X_train_filtered.iloc[idx]
        y_sub = y_train.iloc[idx]
    else:
        X_sub, y_sub = X_train_filtered, y_train

    # 2. Mutual Information
    if task == 'class':
        mi_scores = mutual_info_classif(X_sub, y_sub, random_state=42, n_neighbors=3)
    else:
        mi_scores = mutual_info_regression(X_sub, y_sub, random_state=42, n_neighbors=3)
    
    # 필터링된 컬럼 기준 인덱싱
    mi_idx = np.argsort(mi_scores)[::-1][:top_n]
    mi_features = X_train_filtered.columns[mi_idx].tolist()
    
    # 3. RFE (LGBM)
    estimator = LGBMClassifier(n_estimators=100, random_state=42, verbose=-1) if task == 'class' else LGBMRegressor(n_estimators=100, random_state=42, verbose=-1)
    rfe = RFE(estimator=estimator, n_features_to_select=top_n, step=0.1, verbose=0)
    rfe.fit(X_sub, y_sub)
    rfe_features = X_train_filtered.columns[rfe.support_].tolist()

    # 4. Random Forest Importance
    rf_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1) if task == 'class' else RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_sub, y_sub)
    rf_idx = np.argsort(rf_model.feature_importances_)[::-1][:top_n]
    rf_features = X_train_filtered.columns[rf_idx].tolist()
    
    # 5. Voting
    all_features = mi_features + rfe_features + rf_features
    feature_votes = Counter(all_features)
    selected_features = [feat for feat, _ in feature_votes.most_common(top_n)]
    
    # 부족하면 MI 점수 순으로 채우기
    if len(selected_features) < top_n:
        remaining = top_n - len(selected_features)
        for feat in mi_features:
            if feat not in selected_features:
                selected_features.append(feat)
                remaining -= 1
                if remaining == 0: break
    
    return selected_features, {}
def select_features_multi_target(X_train, y_train, target_type='direction', top_n=20):
    atr_col_name = 'ATR_84'
    if target_type == 'direction':
        selected, stats = select_features_verified(X_train, y_train['next_direction'], task='class', top_n=top_n)
        
        if atr_col_name not in selected and atr_col_name in X_train.columns:
            if len(selected) > 0: selected.pop()
            selected.insert(0, atr_col_name)
            
    print(f"\n[Feature Selection] Top {len(selected)} Features Selected:")
    print(f" -> {', '.join(selected)}")
    return selected, stats


def get_time_decay_weights(dates, decay_rate=0.999):
    """
    오래된 데이터일수록 낮은 가중치(0.5), 최신 데이터일수록 높은 가중치(1.0) 부여
    dates: 날짜 컬럼 (DatetimeIndex 또는 Series)
    """
    n_samples = len(dates)
    # 선형 가중치: 가장 과거 데이터는 0.5, 가장 최신 데이터는 1.0
    weights = np.linspace(0.5, 1.0, n_samples)
    return weights

# [수정 2] process_single_split 함수에 가중치 생성 로직 추가
def process_single_split(split_data, target_type='direction', top_n=20, fold_idx=None, atr_col_name='ATR_84'): 
    train_df = split_data['train'] 
    val_df = split_data['val'] 
    test_df = split_data['test'] 
    fold_type = split_data.get('fold_type', 'unknown')

    print(f"\n{'='*80}")
    print(f" Processing Fold {fold_idx} ({fold_type})")
    print(f"{'='*80}")
    print(f" Train Period: {train_df['date'].min()} ~ {train_df['date'].max()} (N={len(train_df)})")
    print(f" Val   Period: {val_df['date'].min()} ~ {val_df['date'].max()} (N={len(val_df)})")
    print(f" Test  Period: {test_df['date'].min()} ~ {test_df['date'].max()} (N={len(test_df)})")

    target_cols = [
        'next_direction', 'next_log_return', 'next_close', 'next_open', 
        'take_profit_price', 'stop_loss_price', 
        'ATR_84', 'real_entry_price' 
    ]

    train_processed = train_df.dropna(subset=target_cols).reset_index(drop=True)
    val_processed = val_df.dropna(subset=target_cols).reset_index(drop=True)
    test_processed = test_df.dropna(subset=target_cols).reset_index(drop=True)

    exclude_cols = [col for col in target_cols if col != atr_col_name] + ['date']
    feature_cols = [col for col in train_processed.columns if col not in exclude_cols]
    
    X_train = train_processed[feature_cols]
    y_train = train_processed[target_cols]
    X_val = val_processed[feature_cols]
    y_val = val_processed[target_cols]
    X_test = test_processed[feature_cols]
    y_test = test_processed[target_cols]
    
    train_weights = get_time_decay_weights(train_processed['date'])

    balance = y_train['next_direction'].value_counts(normalize=True).to_dict()
    print(f"[Class Balance] Train Set: {balance}")

    # Feature Selection (여기서는 가중치 안 쓰고 원본 데이터 사용)
    selected_features, selection_stats = select_features_multi_target(
        X_train, y_train, target_type=target_type, top_n=top_n
    )

    X_train_sel = X_train[selected_features]
    X_val_sel = X_val[selected_features]
    X_test_sel = X_test[selected_features]

    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_sel), columns=selected_features)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val_sel), columns=selected_features)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test_sel), columns=selected_features)

    return {
        'train': {
            'X': X_train_scaled, 
            'y': y_train.reset_index(drop=True), 
            'dates': train_processed['date'].reset_index(drop=True),
            'weights': train_weights  
        },
        'val': {
            'X': X_val_scaled, 
            'y': y_val.reset_index(drop=True), 
            'dates': val_processed['date'].reset_index(drop=True)
        },
        'test': {
            'X': X_test_scaled, 
            'y': y_test.reset_index(drop=True), 
            'dates': test_processed['date'].reset_index(drop=True)
        },
        'scaler': scaler, 
        'selected_features': selected_features,
        'stats': {
            'fold_idx': fold_idx if fold_idx is not None else split_data.get('fold_idx', 0),
            'fold_type': split_data.get('fold_type', 'unknown')
        }
    }

def split_walk_forward_method(df, train_start_date, final_test_start='2025-01-01', 
                              initial_train_size=800, val_size=150, test_size=150, 
                              step=150, gap_size=30):
    
    df_period = df[df['date'] >= pd.to_datetime(train_start_date)].copy()
    df_period = df_period.sort_values('date').reset_index(drop=True)
    
    if isinstance(final_test_start, str):
        final_test_start = pd.to_datetime(final_test_start)
    
    total_len = len(df_period)
    folds = []
    

    current_test_end = total_len
    
    # Final Holdout 제외한 일반 Fold 생성
    while True:
        test_end_idx = current_test_end
        test_start_idx = test_end_idx - test_size
        val_end_idx = test_start_idx - gap_size
        val_start_idx = val_end_idx - val_size
        train_end_idx = val_start_idx - gap_size
        train_start_idx = train_end_idx - initial_train_size
        
        # 더 이상 훈련 데이터를 확보할 수 없으면 중단
        if train_start_idx < 0: 
            break
        
        train_fold = df_period.iloc[train_start_idx:train_end_idx].copy()
        val_fold = df_period.iloc[val_start_idx:val_end_idx].copy()
        test_fold = df_period.iloc[test_start_idx:test_end_idx].copy()
        
        folds.append({
            'train': train_fold,
            'val': val_fold,
            'test': test_fold,
            'fold_type': 'walk_forward_rolling_reverse'
        })
        
        current_test_end = current_test_end - step
        
        # 너무 과거로 가면 중단 (안전장치)
        if current_test_end < (initial_train_size + val_size + gap_size):
            break
    
    folds.reverse() # 시간순 정렬
    
    # Fold Index 부여
    for idx, fold in enumerate(folds):
        fold['fold_idx'] = idx + 1
        
    # Final Holdout 추가 (별도 처리)
    final_test_df = df_period[df_period['date'] >= final_test_start].copy()
    if len(final_test_df) > 0:
        pre_final_df = df_period[df_period['date'] < final_test_start].copy()
        if len(pre_final_df) >= (initial_train_size + val_size + gap_size):
            final_val_end_idx = len(pre_final_df)
            final_val_start_idx = final_val_end_idx - val_size
            final_train_end_idx = final_val_start_idx - gap_size
            final_train_start_idx = final_train_end_idx - initial_train_size
            
            final_train_data = pre_final_df.iloc[final_train_start_idx:final_train_end_idx].copy()
            final_val_data = pre_final_df.iloc[final_val_start_idx:final_val_end_idx].copy()
            
            folds.append({
                'train': final_train_data,
                'val': final_val_data,
                'test': final_test_df,
                'fold_idx': len(folds) + 1,
                'fold_type': 'final_holdout'
            })
            
    return folds


def build_complete_pipeline_corrected(df_raw, df_hour, train_start_date, **kwargs): 
    print(f"\n{'='*60}")
    print(f" Pipeline Started... (Train Start: {train_start_date})")
    print(f"{'='*60}")

    # 파라미터 처리 로직 (동일)
    lookahead = kwargs.get('lookahead', 30)
    profit_mult = kwargs.get('profit_mult', 2.0)
    stop_mult = kwargs.get('stop_mult', 1.0)

    # 타겟 생성 (동일)
    df = create_targets_4h(df_raw, df_hour,
        lookahead=lookahead, 
        profit_mult=profit_mult, 
        stop_mult=stop_mult
    )

    # 데이터 상태 로그 (동일)
    original_len = len(df_raw)
    remaining_len = len(df)
    drop_ratio = (original_len - remaining_len) / original_len * 100
    print(f"  [Data Status] Original: {original_len} -> Valid: {remaining_len} (Dropped {drop_ratio:.1f}%)")
    
    # 피처 정리 (동일)
    df = remove_raw_prices_and_transform(df, 'direction', 'tvt')
    
    # Split 설정 (중요: train_days 사용)
    initial_train_days = kwargs.get('train_days', 720) # 2년 권장
    initial_train_size = initial_train_days * 6
    
    val_size = 90 * 6
    test_size = 90 * 6
    gap_size = lookahead + 5
    step_size = test_size 

    splits = split_walk_forward_method(
        df, 
        train_start_date=train_start_date,
        final_test_start=kwargs.get('final_test_start', '2025-01-01'),
        initial_train_size=initial_train_size,
        val_size=val_size,
        test_size=test_size,
        step=step_size, 
        gap_size=gap_size
    )
    
    print(f"  [Split Result] Total {len(splits)} folds generated.")

    result = []
    for i, fold in enumerate(splits):
        if len(fold['train']) < (initial_train_size * 0.5):
            print(f"  [Skip] Fold {fold.get('fold_idx')} skipped.")
            continue
            
        res = process_single_split(
            fold, 
            top_n=kwargs.get('top_n', 20), 
            fold_idx=fold['fold_idx'],
            atr_col_name='ATR_84'
        )
        result.append(res)

    return result

In [4]:
import numpy as np
import pandas as pd
import optuna
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score, brier_score_loss
from sklearn.utils.class_weight import compute_class_weight
from sklearn.calibration import CalibratedClassifierCV

class DirectionModels:
    
    @staticmethod
    def get_class_weights(y_train):
        classes = np.unique(y_train)
        weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
        return dict(zip(classes, weights))

    @staticmethod
    def lightgbm(X_train, y_train, X_val, y_val, sample_weight=None):
        optuna.logging.set_verbosity(optuna.logging.WARNING)

        def objective(trial):
            params = {
                'n_estimators': 2000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 80),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'min_child_samples': trial.suggest_int('min_child_samples', 30, 200),
                'subsample': trial.suggest_float('subsample', 0.6, 0.85),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
                'objective': 'binary',
                'metric': 'binary_logloss',
                'class_weight': 'balanced',
                'verbosity': -1,
                'n_jobs': -1,
                'random_state': 42
            }
            
            model = lgb.LGBMClassifier(**params)
            # Early Stopping 30으로 강화
            callbacks = [lgb.early_stopping(stopping_rounds=30, verbose=False)]
            
            # [가중치 적용]
            model.fit(X_train, y_train, sample_weight=sample_weight, 
                      eval_set=[(X_val, y_val)], callbacks=callbacks)
            
            # [복합 지표] LogLoss + AUC (Calibration 고려)
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            logloss = log_loss(y_val, y_pred_proba)
            auc = roc_auc_score(y_val, y_pred_proba)
            
            # LogLoss는 작을수록 좋고, AUC는 클수록 좋음 -> 복합 점수(Minimize)
            return logloss - (0.3 * auc)

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=30) # 30회로 증가
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 2000, 'objective': 'binary', 'metric': 'binary_logloss', 
            'class_weight': 'balanced', 'verbosity': -1, 'n_jobs': -1, 'random_state': 42
        })
        
        final_model = lgb.LGBMClassifier(**best_params)
        final_model.fit(X_train, y_train, sample_weight=sample_weight,
                        eval_set=[(X_val, y_val)], 
                        callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)])
        

        calibrated_model = CalibratedClassifierCV(final_model, method='sigmoid', cv='prefit')
        calibrated_model.fit(X_val, y_val)
        
        return calibrated_model

    @staticmethod
    def xgboost(X_train, y_train, X_val, y_val, sample_weight=None):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        neg, pos = np.bincount(y_train.astype(int))
        scale_pos_weight = neg / pos

        def objective(trial):
            params = {
                'n_estimators': 2000,
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 8),
                'min_child_weight': trial.suggest_int('min_child_weight', 30, 150),
                'subsample': trial.suggest_float('subsample', 0.6, 0.85),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.85),
                'gamma': trial.suggest_float('gamma', 0.5, 5.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
                'scale_pos_weight': scale_pos_weight,
                'objective': 'binary:logistic',
                'eval_metric': 'logloss',
                'tree_method': 'hist',
                'early_stopping_rounds': 30, # 30으로 강화
                'random_state': 42,
                'n_jobs': -1
            }
            
            model = xgb.XGBClassifier(**params)
            # [가중치 적용]
            model.fit(X_train, y_train, sample_weight=sample_weight,
                      eval_set=[(X_val, y_val)], verbose=False)
            
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            logloss = log_loss(y_val, y_pred_proba)
            auc = roc_auc_score(y_val, y_pred_proba)
            return logloss - (0.3 * auc)

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=30)
        
        best_params = study.best_params
        best_params.update({
            'n_estimators': 2000, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 
            'tree_method': 'hist', 'scale_pos_weight': scale_pos_weight,
            'early_stopping_rounds': 30, 'random_state': 42, 'n_jobs': -1
        })
        
        final_model = xgb.XGBClassifier(**best_params)
        final_model.fit(X_train, y_train, sample_weight=sample_weight,
                        eval_set=[(X_val, y_val)], verbose=False)
        
        # [Calibration]
        calibrated_model = CalibratedClassifierCV(final_model, method='sigmoid', cv='prefit')
        calibrated_model.fit(X_val, y_val)
        
        return calibrated_model

    @staticmethod
    def catboost(X_train, y_train, X_val, y_val, sample_weight=None):
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        
        def objective(trial):
            params = {
                'iterations': 2000,
                'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.02, log=True), # 더 낮춤
                'depth': trial.suggest_int('depth', 4, 6), # 7도 깊음. 4~6 권장
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 5, 30), # 규제 하한선 올림
                'random_strength': trial.suggest_float('random_strength', 1, 10),
                'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
                
                'rsm': trial.suggest_float('rsm', 0.5, 0.9), # 1.0 제외 (무조건 피처 일부는 가림)

                'loss_function': 'Logloss', 
                'eval_metric': 'Logloss',
                'auto_class_weights': 'Balanced',
                'logging_level': 'Silent',
                'random_seed': 42,
                'od_type': 'Iter',
                'od_wait': 50, 
                'allow_writing_files': False
            }
            
            model = cb.CatBoostClassifier(**params)
            model.fit(X_train, y_train, sample_weight=sample_weight, eval_set=(X_val, y_val))
            
            y_pred_proba = model.predict_proba(X_val)[:, 1]
            logloss = log_loss(y_val, y_pred_proba)
            auc = roc_auc_score(y_val, y_pred_proba)
            
            # AUC 가중치를 조금 더 높여서 순위(Rank) 학습에 집중
            return logloss - (0.5 * auc) 

        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=30)
        
        best_params = study.best_params
        best_params.update({
            'iterations': 2000, 
            'loss_function': 'Logloss', 
            'eval_metric': 'Logloss',
            'auto_class_weights': 'Balanced', 
            'logging_level': 'Silent',
            'random_seed': 42, 
            'od_type': 'Iter', 
            'od_wait': 50, 
            'allow_writing_files': False
        })
        
        final_model = cb.CatBoostClassifier(**best_params)
        final_model.fit(X_train, y_train, sample_weight=sample_weight, eval_set=(X_val, y_val))
        
        calibrated_model = CalibratedClassifierCV(final_model, method='isotonic', cv='prefit')
        calibrated_model.fit(X_val, y_val)
        
        return calibrated_model



ML_MODELS_CLASSIFICATION = [
    {'index': 1, 'name': 'CatBoost', 'func': DirectionModels.catboost, 'needs_val': True},
    {'index': 2, 'name': 'LightGBM', 'func': DirectionModels.lightgbm, 'needs_val': True},
    {'index': 3, 'name': 'XGBoost', 'func': DirectionModels.xgboost, 'needs_val': True}
]


In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pandas as pd
import joblib
import json
import os
import gc
import traceback
import shap  # SHAP 추가
from datetime import datetime

class ModelEvaluator:
    def __init__(self, save_models=False):
        self.results = []
        self.best_thresholds = {} # 모델별 최적 Threshold 저장
        self.save_models = save_models
        self.models = {} if save_models else None
        self.prediction_logs = {} # 상세 예측 로그
        
    def optimize_threshold(self, y_true, buy_prob, min_trades=20, reward_risk_ratio=1.5):
        """
        [수정됨] 수수료(Fee)를 반영한 실질 수익(Net Expectancy) 최적화
        """
        best_score = -999.0
        best_th = 0.5
        
        # 수수료 보정값 (보수적으로 0.15R 정도 깐다고 가정 - 진입/청산/슬리피지 포함)
        FEE_PENALTY_R = 0.15 
        
        # 탐색 범위를 조금 더 넓힘 (40% ~ 99%)
        percentiles = np.arange(40, 99, 2)
        
        for pct in percentiles:
            th = np.percentile(buy_prob, pct)
            if th < 0.5: continue 
            
            preds = (buy_prob >= th).astype(int)
            n_trades = preds.sum()
            
            if n_trades < min_trades: continue
            
            wins = ((preds == 1) & (y_true == 1)).sum()
            losses = n_trades - wins
            win_rate = wins / n_trades

            # 승률 40% 미만은 가차없이 탈락 (안정성)
            if win_rate < 0.4: continue
            

            real_expectancy = (win_rate * (reward_risk_ratio - FEE_PENALTY_R)) - ((1 - win_rate) * (1.0 + FEE_PENALTY_R))
            
            
            if real_expectancy <= 0:
                score = -99.0
            else:
                score = real_expectancy 
            
            if score > best_score:
                best_score = score
                best_th = th
                
        return max(best_th, 0.5)

    def evaluate_model(self, model, X_train, y_train, X_val, y_val, X_test, y_test, 
                       model_name, profit_mult=1.5, stop_mult=1.0,
                       test_dates=None, test_prices=None, y_test_df=None):  
        

        rr_ratio = profit_mult / stop_mult if stop_mult > 0 else 1.5
        fee = 0.0015 
        

        FEE_R = 0.15 

        # 1. 확률 예측
        tr_prob = model.predict_proba(X_train)[:, 1]
        val_prob = model.predict_proba(X_val)[:, 1]
        te_prob = model.predict_proba(X_test)[:, 1]
        
        # 2. Threshold 최적화
        min_trades_val = max(10, int(len(y_val) * 0.01)) 
        best_th = self.optimize_threshold(y_val.astype(int), val_prob, min_trades=min_trades_val, reward_risk_ratio=rr_ratio)
        self.best_thresholds[model_name] = best_th

        # 3. 지표 계산 함수 (MDD 추가됨)
        def calculate_metrics(y_true, probs, threshold, prefix):
            preds = (probs >= threshold).astype(int)
            n_trades = np.sum(preds == 1)
            
            acc = accuracy_score(y_true, preds)
            prec = precision_score(y_true, preds, zero_division=0)
            
            mdd = 0.0
            total_return_r = 0.0
            consecutive_loss = 0
            max_consecutive_loss = 0
            
            if n_trades > 0:
                # 승/패 마스크
                is_win = (preds == 1) & (y_true == 1)
                is_loss = (preds == 1) & (y_true == 0)
                
                wins = np.sum(is_win)
                win_rate = wins / n_trades
                
                # Net Expectancy (R 단위)
                net_exp = (win_rate * (rr_ratio - FEE_R)) - ((1 - win_rate) * (1.0 + FEE_R))
                
                # --- [추가] MDD 및 Equity 계산 (R 단위 시뮬레이션) ---
                # 거래가 발생한 순서대로 PnL 배열 생성
                trade_pnl = []
                for i in range(len(preds)):
                    if preds[i] == 1:
                        if y_true[i] == 1:
                            pnl = rr_ratio - FEE_R
                            trade_pnl.append(pnl)
                            consecutive_loss = 0
                        else:
                            pnl = -1.0 - FEE_R
                            trade_pnl.append(pnl)
                            consecutive_loss += 1
                            max_consecutive_loss = max(max_consecutive_loss, consecutive_loss)
                
                trade_pnl = np.array(trade_pnl)
                cumulative_pnl = np.cumsum(trade_pnl)
                total_return_r = cumulative_pnl[-1] if len(cumulative_pnl) > 0 else 0
                
                # MDD 계산
                running_max = np.maximum.accumulate(cumulative_pnl)
                drawdown = running_max - cumulative_pnl
                mdd = np.max(drawdown) if len(drawdown) > 0 else 0.0
                
            else:
                win_rate, net_exp = 0.0, 0.0
            
            # 요약 딕셔너리 리턴
            return {
                f"{prefix}_Acc": acc, 
                f"{prefix}_Prec": prec,
                f"{prefix}_Exp": net_exp, 
                f"{prefix}_WinRate": win_rate, 
                f"{prefix}_Trades": n_trades,
                f"{prefix}_MDD_R": mdd,          
                f"{prefix}_Total_R": total_return_r,
                f"{prefix}_MaxSeqLoss": max_consecutive_loss 
            }
            
        metrics_train = calculate_metrics(y_train.astype(int), tr_prob, best_th, "Train")
        metrics_val = calculate_metrics(y_val.astype(int), val_prob, best_th, "Val")
        metrics_test = calculate_metrics(y_test.astype(int), te_prob, best_th, "Test") 
        
        result = {
            'Model': model_name,
            'Threshold': best_th,
            **metrics_train,
            **metrics_val,
            **metrics_test
        }
        
        # 로그 출력 보강
        print(f"    -> [EVAL] {model_name:<12} | Test_Exp: {result['Test_Exp']:.3f}R | MDD: {result['Test_MDD_R']:.2f}R | Trades: {result['Test_Trades']}")

        # 5. 상세 로그 저장 (전략팀이 엑셀 돌리기 편하게 PnL 컬럼 추가)
        if test_dates is not None and y_test_df is not None:
            # ... (기존 데이터 추출 로직 동일) ...
            cols_to_extract = ['take_profit_price', 'stop_loss_price', 'real_entry_price']
            extracted_data = {}
            if isinstance(y_test_df, pd.DataFrame):
                for col in cols_to_extract:
                    extracted_data[col] = y_test_df[col].values if col in y_test_df.columns else np.zeros(len(te_prob))
            else:
                 for col in cols_to_extract: extracted_data[col] = np.zeros(len(te_prob))

            if hasattr(test_prices, 'columns') and 'ETH_Close' in test_prices.columns:
                close_vals = test_prices['ETH_Close'].values
            else:
                close_vals = np.zeros(len(te_prob))

            min_len = min(len(test_dates), len(te_prob))
            
            # 데이터프레임 생성
            pred_df = pd.DataFrame({
                'timestamp': test_dates[:min_len],
                'close': close_vals[:min_len],
                'prob': te_prob[:min_len],
                'threshold': [best_th]*min_len,
                'signal': (te_prob[:min_len] >= best_th).astype(int),
                'actual_target': y_test[:min_len].astype(int),
                **{k: v[:min_len] for k, v in extracted_data.items()}
            })
            
            # [중요] 전략팀용 PnL 컬럼 추가 (R unit)
            # signal이 1일 때 -> 이기면 (RR - Fee), 지면 (-1 - Fee), 아니면 0
            # 여기서 '지면'은 0을 의미 (이전 단계에서 횡보를 제거했으므로 0=SL 가정)
            pred_df['strategy_pnl_R'] = 0.0
            
            # 벡터 연산으로 PnL 계산
            mask_signal = pred_df['signal'] == 1
            mask_win = (pred_df['actual_target'] == 1)
            
            # 승리 시
            pred_df.loc[mask_signal & mask_win, 'strategy_pnl_R'] = rr_ratio - FEE_R
            # 패배 시
            pred_df.loc[mask_signal & (~mask_win), 'strategy_pnl_R'] = -1.0 - FEE_R
            
            # 누적 PnL (Equity Curve용)
            pred_df['cumulative_pnl_R'] = pred_df['strategy_pnl_R'].cumsum()

            if not isinstance(pred_df.index, pd.DatetimeIndex):
                pred_df.set_index('timestamp', inplace=True)
                
            self.prediction_logs[model_name] = pred_df

        self.results.append(result)
        if self.save_models: self.models[model_name] = model
        
        return result

    
    def get_summary_dataframe(self): return pd.DataFrame(self.results)
    def get_models_dict(self): return self.models or {}
    def get_prediction_logs(self): return self.prediction_logs
    def get_best_thresholds(self): return self.best_thresholds


class ModelTrainer:
    def __init__(self, evaluator):
        self.evaluator = evaluator
    
    def _prepare_target(self, y_data):
        if isinstance(y_data, pd.DataFrame):
            y_vals = y_data.iloc[:, 0].values
        elif isinstance(y_data, pd.Series):
            y_vals = y_data.values
        else:
            y_vals = y_data
            
        y_vals = np.array(y_vals).flatten()
        y_vals = np.nan_to_num(y_vals, nan=0.0)
        return np.round(y_vals).astype(int)

    # [수정] sample_weight 인자 추가
    def train_all_models(self, X_train, y_train, X_val, y_val, X_test, y_test, 
                         profit_mult, stop_mult, ml_models,
                         test_dates=None, test_prices=None, 
                         sample_weight=None): # <--- 여기 추가됨
        
        y_train_arr = self._prepare_target(y_train)
        y_val_arr = self._prepare_target(y_val)
        y_test_arr = self._prepare_target(y_test)
        
        real_rr = profit_mult / stop_mult if stop_mult > 0 else 0
        print(f"  [Info] Training Models... (Target RR: {real_rr:.2f})")
        
        for config in ml_models:
            try:
                # [수정] 모델 함수 호출 시 sample_weight 전달
                # config['func']는 DirectionModels.lightgbm 등을 가리킴
                if config.get('needs_val', False):
                    model = config['func'](
                        X_train, y_train_arr, 
                        X_val, y_val_arr, 
                        sample_weight=sample_weight # <--- 전달!
                    )
                else:

                    model = config['func'](
                        X_train, y_train_arr, 
                        sample_weight=sample_weight # <--- 전달!
                    )
                
                self.evaluator.evaluate_model(
                    model=model, 
                    X_train=X_train, y_train=y_train_arr, 
                    X_val=X_val, y_val=y_val_arr, 
                    X_test=X_test, y_test=y_test_arr,         
                    model_name=config['name'], 
                    profit_mult=profit_mult, 
                    stop_mult=stop_mult,
                    test_dates=test_dates, 
                    test_prices=test_prices,
                    y_test_df=y_test 
                )
                
                del model
                gc.collect()
                
            except Exception as e:
                print(f"[Error] Failed {config['name']}: {e}")
                traceback.print_exc() # 에러 로그 너무 길면 주석 처리


def save_shap_feature_importance(model, X_val, feature_names, save_path, n_sample=1500):
    if isinstance(X_val, pd.DataFrame):
        X_use = X_val.sample(n=min(n_sample, len(X_val)), random_state=42)
    else:
        X_use = X_val[:n_sample]
    
    try:
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_use)
        if isinstance(shap_values, list):  
            shap_values = shap_values[1]
        vals = X_use.values if isinstance(X_use, pd.DataFrame) else X_use

        scores = []
        for i, name in enumerate(feature_names):
            imp = np.mean(np.abs(shap_values[:, i]))
            if np.std(vals[:, i]) > 1e-9:
                direction = np.corrcoef(vals[:, i], shap_values[:, i])[0, 1]
            else:
                direction = 0.0
            scores.append({"feature": name, "importance": imp, "direction": direction})
        pd.DataFrame(scores).sort_values("importance", ascending=False).to_csv(save_path, index=False)
    except Exception as e:
        print(f" [Warning] SHAP failed: {e}")


def save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data, result_dir):
    base_dir = f"{result_dir}/{trial_name}/fold_{fold_idx}_{fold_type}"
    os.makedirs(base_dir, exist_ok=True)
    
    scaler = fold_data.get('scaler')
    if scaler is not None:
        joblib.dump(scaler, f"{base_dir}/scaler.pkl")
        
    summary = evaluator.get_summary_dataframe()
    if not summary.empty:
        summary.to_csv(f"{base_dir}/fold_summary.csv", index=False)
    
    pred_logs = evaluator.get_prediction_logs()
    for model_name, df_log in pred_logs.items():
        df_log.to_csv(f"{base_dir}/predictions_{model_name}.csv")

    feature_names = fold_data.get('selected_features', [])
    if hasattr(feature_names, 'tolist'):
        feature_names = feature_names.tolist()
    
    model_params_info = {}
    best_thresholds = evaluator.get_best_thresholds()

    for name, model in evaluator.get_models_dict().items():
        deployment_artifact = {
            'model': model,
            'scaler': scaler,
            'features': feature_names,
            'threshold': best_thresholds.get(name, 0.5),
            'model_name': name,
            'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        joblib.dump(deployment_artifact, f"{base_dir}/model_{name}_artifact.pkl")

        # [수정] 안전한 원본 모델 추출 로직
        base_model = model
        
        # 1. CalibratedClassifierCV인 경우
        if hasattr(model, 'calibrated_classifiers_') and len(model.calibrated_classifiers_) > 0:
            calibrated_clf = model.calibrated_classifiers_[0]
            if hasattr(calibrated_clf, 'base_estimator'):
                base_model = calibrated_clf.base_estimator
            elif hasattr(calibrated_clf, 'estimator'):
                base_model = calibrated_clf.estimator
        
        # 2. 일반 래퍼나 구버전 호환
        elif hasattr(model, 'base_estimator'):
            base_model = model.base_estimator
        elif hasattr(model, 'estimator'):
            base_model = model.estimator
        
        try:
            if hasattr(base_model, 'get_params'):
                model_params_info[name] = base_model.get_params()
            elif hasattr(base_model, 'get_all_params'): 
                model_params_info[name] = base_model.get_all_params()
        except:
            model_params_info[name] = "Could not extract params"

        if (hasattr(base_model, "feature_importances_") or hasattr(base_model, "get_feature_importance")) and len(feature_names) > 0:
            shap_path = f"{base_dir}/shap_importance_{name}.csv"
            save_shap_feature_importance(
                model=base_model, 
                X_val=fold_data['val']['X'], 
                feature_names=feature_names,
                save_path=shap_path
            )

    meta_data = {
        'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'fold_idx': fold_idx,
        'best_thresholds': best_thresholds,
        'features': feature_names,
        'params': {
            'profit_mult': fold_data.get('profit_mult'), 
            'stop_mult': fold_data.get('stop_mult'),     
            'lookahead': fold_data.get('lookahead'),
            'top_n': fold_data.get('top_n'),
            'train_days': fold_data.get('train_days')
        },
        'model_hyperparams': str(model_params_info)
    }
    
    with open(f"{base_dir}/metadata.json", 'w') as f:
        json.dump(meta_data, f, indent=4)
            
    return summary


In [6]:
def run_optuna_optimization(df_merged, df_hour, ml_models, n_trials=30):
    
    TIMESTAMP = datetime.now().strftime("%Y-%m-%d")
    RESULT_DIR = f"model_results/{TIMESTAMP}_횡보장제거_Fix"
    os.makedirs(RESULT_DIR, exist_ok=True)
    
    LOG_PATH = f"{RESULT_DIR}/optuna_logs.csv"

    # 빠른 조회를 위한 Lookup 테이블 생성
    ohlc_cols = ['date', 'ETH_Open', 'ETH_High', 'ETH_Low', 'ETH_Close'] 
    lookup_df = df_merged[ohlc_cols].copy()
    lookup_df['date'] = pd.to_datetime(lookup_df['date'])
    lookup_df = lookup_df.set_index('date').sort_index()
    
    # 기존 로그 로드 (중단된 지점부터 다시 시작 기능)
    existing_history = pd.DataFrame()
    if os.path.exists(LOG_PATH):
        try:
            existing_history = pd.read_csv(LOG_PATH)
            print(f"\n[Resume] Loaded {len(existing_history)} existing trials.")
        except: 
            print("[Warning] Failed to load existing log. Starting fresh.")

    # CSV 헤더 초기화
    if not os.path.exists(LOG_PATH):
        with open(LOG_PATH, "w") as f:
            f.write("trial,lookahead,profit_mult,stop_mult,top_n,train_days,score\n")

    def objective(trial):
        nonlocal existing_history 
        
        # 1. 하이퍼파라미터 제안
        # 1. Train Days: 너무 길면 옛날(2년 전) 장세까지 배워서 오히려 안 좋습니다. 최신 트렌드 반영.
        train_days = trial.suggest_int('train_days', 180, 540, step=90) 


        s_mult = trial.suggest_float('stop_mult', 1.0, 1.8, step=0.2)


        p_mult = trial.suggest_float('profit_mult', 1.5, 2.5, step=0.25)

        # 4. Lookahead: 4시간봉 스윙이면 24시간(6봉) ~ 72시간(18봉)이 적당합니다.
        # 30봉(5일)은 너무 깁니다. 예측력이 떨어집니다.
        lookahead = trial.suggest_int('lookahead', 6, 18, step=6) 

        # 5. Top N: 너무 적으면 정보 부족, 너무 많으면 과적합. 20~50 정도 추천.
        top_n = trial.suggest_int('top_n', 20, 50, step=5)
        # 2. Pruning (비현실적 조합 제거)
        if p_mult > 2.5 and lookahead < 12:
            raise optuna.TrialPruned("Unrealistic: High TP with Short Lookahead")

        # 3. 중복 실행 방지 체크
        if not existing_history.empty:
            # 모든 파라미터가 일치하는지 확인
            mask = (
                (existing_history['lookahead'] == lookahead) &
                (np.isclose(existing_history['profit_mult'], p_mult, atol=1e-5)) &
                (np.isclose(existing_history['stop_mult'], s_mult, atol=1e-5)) &
                (existing_history['top_n'] == top_n) &
                (existing_history['train_days'] == train_days)
            )
            if mask.any():
                prev_score = existing_history.loc[mask, 'score'].values[0]
                print(f"[Skip] Already done. Score: {prev_score}")
                return prev_score

        trial_name = f"T{trial.number}_L{lookahead}_P{p_mult:.1f}_S{s_mult:.1f}_N{top_n}_D{train_days}"
        print(f"\n{'='*60}\n Starting {trial_name}\n{'='*60}")
        
        try:
            # 4. 파이프라인 실행 (데이터 생성 및 분할)
            pipeline_result = build_complete_pipeline_corrected(
                df_raw=df_merged,
                df_hour=df_hour,
                train_start_date='2020-01-01',
                final_test_start='2025-01-01',
                lookahead_periods=lookahead, 
                lookahead=lookahead,          
                profit_mult=p_mult,
                stop_mult=s_mult,
                top_n=top_n,
                train_days=train_days         
            )
            
            fold_scores = []
            
            for fold_data in pipeline_result:
                stats = fold_data.get('stats', {}) 
                fold_idx = stats.get('fold_idx', 0)
                fold_type = stats.get('fold_type', 'unknown')
                
                print(f"   >> Running Fold {fold_idx} ({fold_type})")
                
                fold_data['profit_mult'] = p_mult
                fold_data['stop_mult'] = s_mult
                fold_data['lookahead'] = lookahead 
                fold_data['top_n'] = top_n           # 추가됨
                fold_data['train_days'] = train_days # 추가됨 (이게 빠져 있었음)

                test_dates = pd.to_datetime(fold_data['test']['dates'])
                test_ohlc = lookup_df.reindex(test_dates).fillna(0)
                
                evaluator = ModelEvaluator(save_models=True) 
                trainer = ModelTrainer(evaluator)
                
                trainer.train_all_models(
                    fold_data['train']['X'], fold_data['train']['y'],
                    fold_data['val']['X'], fold_data['val']['y'],
                    fold_data['test']['X'], fold_data['test']['y'],
                    p_mult, s_mult, ml_models,
                    test_dates=test_dates, 
                    test_prices=test_ohlc
                )

                # 결과 저장 (metadata.json에 위에서 넣은 정보들이 포함됨)
                summary = save_fold_results(fold_idx, fold_type, evaluator, trial_name, fold_data, RESULT_DIR)

                if 'Test_Exp' in summary.columns:
                    best_fold_score = summary['Test_Exp'].max()
                    fold_scores.append(best_fold_score)
                
                del evaluator, trainer
                gc.collect()
            
            final_score = np.mean(fold_scores) if fold_scores else -99.0
            print(f"\n === Trial Score: {final_score:.4f}R ===")
            
            # 5. 로그 파일 기록 (train_days 포함)
            with open(LOG_PATH, "a") as f:
                f.write(f"{trial.number},{lookahead},{p_mult},{s_mult},{top_n},{train_days},{final_score}\n")
            
            # 메모리 상의 히스토리 업데이트
            new_row = pd.DataFrame([[trial.number, lookahead, p_mult, s_mult, top_n, train_days, final_score]], 
                                   columns=['trial','lookahead','profit_mult','stop_mult','top_n','train_days','score'])
            
            if existing_history.empty: 
                existing_history = new_row
            else: 
                existing_history = pd.concat([existing_history, new_row], ignore_index=True)
                
            return final_score
            
        except Exception as e:
            print(f" [Error] Trial Failed: {e}")
            traceback.print_exc()
            return -99.0

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=n_trials)
    
    print(f"\n[Optuna] Best Params: {study.best_params}")
    return study



In [ ]:
# -----------------------------------------------------------------------------
# 5. Main Execution Block
# -----------------------------------------------------------------------------
df_merged=pd.read_csv("eth_4hour_all.csv")
df_hour = pd.read_csv("eth_hour.csv")
# [실제 실행]
study = run_optuna_optimization(df_merged, df_hour, ML_MODELS_CLASSIFICATION, n_trials=100)

print("==================================================")
print(f" Best Expectancy: {study.best_value:.4f}")
print("==================================================")



[I 2025-12-03 00:09:58,078] A new study created in memory with name: no-name-ddda9605-f178-4a89-ad30-30add95841b9



 Starting T0_L30_P1.9_S1.4_N35_D630

 🚀 Pipeline Started... (Train Start: 2020-01-01)
Valid Samples: 12509/17895 (Dropped: 5386)
  [Data Status] Original: 17925 -> Valid: 12509 (Dropped 30.2%)
[Feature Cleaning] Dropping 438 columns.
  - Altcoin Metrics: 418 (DOGE, XRP, etc.)
  - Raw Prices/Volumes: 20
  [Split Result] Total 15 folds generated.

 Processing Fold 1 (walk_forward_rolling_reverse)
 Train Period: 2020-04-03 09:00:00 ~ 2022-01-12 05:00:00 (N=3780)
 Val   Period: 2022-01-18 09:00:00 ~ 2022-04-20 01:00:00 (N=540)
 Test  Period: 2022-04-26 01:00:00 ~ 2022-07-27 01:00:00 (N=540)
[Class Balance] Train Set: {0: 0.5283068783068783, 1: 0.4716931216931217}
  [Collinearity] Dropping 24 features with correlation > 0.9
Features reduced from 178 to 154 after correlation check.

[Feature Selection] Top 35 Features Selected:
 -> ATR_84, curve-dex_eth_tvl_ma180_ratio, BTC_PRICE_VS_HIGH_360p, SP500_ma180_ratio, ETH_BB_WIDTH, GOLD_ma180_ratio, usdt_eth_mcap_pct_chg_24h, aave_eth_tvl_1d_chg,

  [Collinearity] Dropping 37 features with correlation > 0.9
Features reduced from 178 to 141 after correlation check.

[Feature Selection] Top 35 Features Selected:
 -> ATR_84, fred_RRPONTSYD, optimism_tvl_ma180_ratio, ETH_PRICE_VS_LOW_360p, BTC_MACD_72_156_54_Slow_Pct, ETH_BB_WIDTH, ETH_PRICE_VS_HIGH_120p, usdt_eth_mcap_ma180_ratio, BTC_BB_WIDTH, DXY_ma180_ratio, ETH_RSI_84, makerdao_eth_tvl_1d_chg, fred_WALCL, arbitrum_tvl_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, BTC_ATR_84, FR_Trend, ETH_OBV_Trend_ma180_ratio, ETH_PRICE_VS_HIGH_30p, BTC_ADX_14, BTC_PRICE_VS_LOW_120p, BTC_PRICE_VS_LOW_360p, ETH_MACDH_72_156_54_Slow_Pct, ETH_ADX_14, ETH_PRICE_VS_LOW_30p, ETH_PRICE_VS_LOW_120p, GOLD_ma180_ratio, SP500_ma180_ratio, VIX_ma180_ratio, BTC_Dist_SMA_200_ma180_ratio, ETH_Dist_SMA_120_ma180_ratio, ETH_Dist_SMA_300_ma180_ratio, ETH_Dist_SMA_200_ma180_ratio, usdt_eth_mcap_pct_chg_24h, eth_chain_tvl_1d_chg

 Processing Fold 7 (walk_forward_rolling_reverse)
 Train Period: 2021-10-10 01:00:00 ~

  [Collinearity] Dropping 36 features with correlation > 0.9
Features reduced from 178 to 142 after correlation check.

[Feature Selection] Top 35 Features Selected:
 -> ATR_84, ETH_PRICE_VS_HIGH_360p, BTC_OBV_Trend_ma180_ratio, BTC_PRICE_VS_LOW_360p, ETH_OBV_Trend_ma180_ratio, ETH_PRICE_VS_HIGH_120p, ETH_PRICE_VS_LOW_120p, ETH_BB_WIDTH, BTC_BB_WIDTH, VIX_ma180_ratio, usdt_eth_mcap_pct_chg_24h, zksync_era_tvl_1d_chg, fred_DTWEXBGS, ETH_MACD_72_156_54_Slow_Pct, base_tvl_ma180_ratio, optimism_tvl_ma180_ratio, GOLD_ma180_ratio, ETH_PRICE_VS_HIGH_30p, BTC_ATR_84, BTC_MACD_72_156_54_Slow_Pct, BTC_MACDH_72_156_54_Slow_Pct, BTC_PRICE_VS_LOW_120p, ETH_RSI_84, ETH_MACDH_72_156_54_Slow_Pct, ETH_PRICE_VS_LOW_30p, ETH_BREAKOUT_STR_120p, DXY_ma180_ratio, SP500_ma180_ratio, arbitrum_tvl_ma180_ratio, curve-dex_eth_tvl_ma180_ratio, BTC_Dist_SMA_300_ma180_ratio, ETH_Dist_SMA_300_ma180_ratio, makerdao_eth_tvl_1d_chg, optimism_tvl_1d_chg, lido_eth_tvl_1d_chg

 Processing Fold 13 (walk_forward_rolling_rev

    -> [EVAL] LightGBM     | Test_Exp: -0.266R | MDD: 19.55R | Trades: 40
    -> [EVAL] XGBoost      | Test_Exp: 0.264R | MDD: 20.53R | Trades: 90
   >> Running Fold 11 (walk_forward_rolling_reverse)
  [Info] Training Models... (Target RR: 1.36)
    -> [EVAL] CatBoost     | Test_Exp: 1.207R | MDD: 0.00R | Trades: 1
    -> [EVAL] LightGBM     | Test_Exp: 0.534R | MDD: 3.45R | Trades: 14
    -> [EVAL] XGBoost      | Test_Exp: 0.118R | MDD: 37.95R | Trades: 119
   >> Running Fold 12 (walk_forward_rolling_reverse)
  [Info] Training Models... (Target RR: 1.36)
    -> [EVAL] CatBoost     | Test_Exp: -0.173R | MDD: 51.03R | Trades: 222
    -> [EVAL] LightGBM     | Test_Exp: -0.172R | MDD: 51.29R | Trades: 188
    -> [EVAL] XGBoost      | Test_Exp: -0.276R | MDD: 59.36R | Trades: 205
   >> Running Fold 13 (walk_forward_rolling_reverse)
  [Info] Training Models... (Target RR: 1.36)
    -> [EVAL] CatBoost     | Test_Exp: 1.207R | MDD: 0.00R | Trades: 2
    -> [EVAL] LightGBM     | Test_Exp: -1.1

  [Collinearity] Dropping 30 features with correlation > 0.9
Features reduced from 178 to 148 after correlation check.

[Feature Selection] Top 20 Features Selected:
 -> ATR_84, fred_RRPONTSYD, fred_DTWEXBGS, ETH_MACD_72_156_54_Slow_Pct, BTC_ATR_84, BTC_BB_WIDTH, BTC_MACD_72_156_54_Slow_Pct, ETH_BB_WIDTH, ETH_RSI_84, ETH_PRICE_VS_LOW_360p, ETH_Price_div_VWMA_50d, DXY_ma180_ratio, GOLD_ma180_ratio, optimism_tvl_ma180_ratio, usdt_eth_mcap_ma180_ratio, BTC_Dist_SMA_200_ma180_ratio, ETH_Dist_SMA_200_ma180_ratio, eth_chain_tvl_1d_chg, lido_eth_tvl_1d_chg, usdt_eth_mcap_pct_chg_24h

 Processing Fold 7 (walk_forward_rolling_reverse)
 Train Period: 2021-09-02 21:00:00 ~ 2024-02-23 05:00:00 (N=4860)
 Val   Period: 2024-02-29 05:00:00 ~ 2024-06-19 13:00:00 (N=540)
 Test  Period: 2024-06-26 21:00:00 ~ 2024-10-10 21:00:00 (N=540)
[Class Balance] Train Set: {0: 0.5430041152263374, 1: 0.45699588477366254}
  [Collinearity] Dropping 31 features with correlation > 0.9
Features reduced from 178 to 147 a

    -> [EVAL] XGBoost      | Test_Exp: -1.150R | MDD: 12.65R | Trades: 12
   >> Running Fold 5 (walk_forward_rolling_reverse)
  [Info] Training Models... (Target RR: 1.15)
    -> [EVAL] CatBoost     | Test_Exp: 0.328R | MDD: 8.05R | Trades: 32
    -> [EVAL] LightGBM     | Test_Exp: 0.462R | MDD: 2.30R | Trades: 12
    -> [EVAL] XGBoost      | Test_Exp: 0.228R | MDD: 12.65R | Trades: 39
   >> Running Fold 6 (walk_forward_rolling_reverse)
  [Info] Training Models... (Target RR: 1.15)
    -> [EVAL] CatBoost     | Test_Exp: -0.229R | MDD: 4.60R | Trades: 7
    -> [EVAL] LightGBM     | Test_Exp: 0.000R | MDD: 0.00R | Trades: 0
    -> [EVAL] XGBoost      | Test_Exp: 0.693R | MDD: 2.30R | Trades: 14
   >> Running Fold 7 (walk_forward_rolling_reverse)
  [Info] Training Models... (Target RR: 1.15)
    -> [EVAL] CatBoost     | Test_Exp: -0.198R | MDD: 40.85R | Trades: 131
    -> [EVAL] LightGBM     | Test_Exp: -0.188R | MDD: 128.50R | Trades: 449
    -> [EVAL] XGBoost      | Test_Exp: 0.000R | M